## Math Problem Set 7
Problems 3, 6, 7 and 10 at the end of the Book.

###  Problem 3
For the treated otimization routines list $i)$ the basic idea, $ii)$ the types of problems each one is applicable to, $iii)$ relative strengths and $iv)$ weaknesses for each one of them.

### Newton's Method
can be used as a root finder or minimizer of a function $f$. I'll focus on the root finding version here.

$i)$ Find a tangent $t(x_k)$ to $f(x_k)$ and find the next point by solving $0=t(x_{k+1})$ for $x_{k+1}$. The next point $x_k$ will be closer to thee root of $f$ than the first point $x_{k+1}$. Repeat the steps until convergence.

$ii)$ Newton's is applicable to root finding as well as local minimizing problems to functions that are differentiable at least once or twice, respectively. Moreover it requires positive definiteness the hessian matrix.

$iii)$ Strengths are the fast speed of convergence (quadratic). The simple intution and the simple implementation.

$iv)$ Weaknesses are the strong dependence on the starting point when the function that is to minimize has multiple extrema or roots. It is a priori not possibe to know which one one will end up with. Also differentiability can be problematic in cases with disountinuities. Moreover, when it is hard to findd analytical expressions for the gradient and thee hessian, they must be approximated which can be computationally expensive, especially so if the the dimensionality of the problem is high.


### Gradient Descent

$i)$ First order iterative optimization algorithm for finding the minimum of a function. To find a local minimum, one takes steps proportional to the negative of the gradient of the function at the current point. The parameters of the objective function are updated in the opposite direction of the gradient of the objective function with respect to the parameters.

$ii)$ If the initial guess is far from the optimum, use a gradient descent method for several steps to get closer to the optimum and then switch back to Newton or BFGS methods.

$iii)$ Very simple to implement 

$iv)$ Calculating the gradient can be very expensive if the size of the data is large. Knowing how far to travel along the gradient can be problematic if the objective function is non-convex.  It is also possible to be stuck in local optimums. Can be very slow to converge in practice. For non-differentiable functions, gradient methods are often ill-defined.


### Conjugate Gradient

$i)$ For iteration, construct a number of conjugate directions of the gradient of a function $f$ and move in these directions one after another as opposed to steepest gradient decent where the algorithm moves in direction of gradient in one single step. Within one iteration the Gram-Schmidt algorithm can be used to find the orthogonal directions. 

$ii)$ It is best suited for minimization of quadratic functions. Also, obviously the gradient must exist. 

$iii)$ As it only moves foreward in one direction at a time the computaional cost for one such step are lower than for example one teration of Newton Method. Moreover, the algorithm can terminate before all conjugate directions have been processed. Als, there exist modifications that make thee algorithm applicable to problems of higher order than quadratic.

$iv)$ The speed of convergence depends on the condition number of the matrices in $f$. For some problems he algorithm may converge faster than for others.

### BFGS

$i)$ From an initial guess $x_{0}$ and an approximate Hessian matrix $B_{0}$, first obtain a direction $p_{k}$ by solving $B_{k}p_{k} = - \triangledown f(x_{k})$. Perform a line search to find an acceptable step size $ \alpha_{k}$ in the direction found in the first step, such that $\alpha_{k} = argmin f(x_{k} + \alpha p_{k})$. Set $s_{k} = \alpha_{k} p_{k}$ and update $x_{k+1} = x_{k} + s_{k}$.  Then $y_{k} = \triangledown f(x_{k+1}) - \triangledown f(x_{k})$. Finally, $B_{k+1} = B_{k} + \frac{y_{k}y_{k}^{T}}{y_{k}^{T}s_{k}} - \frac{B_{k}s_{k}s_{k}^{T}B_{k}}{s_{k}^{T}B_{k}s_{k}}$. The algorithm is repeated until the norm of the gradient converges.

$ii)$Requires gradient to be zero for optimality. Not guaranteed to converge unless the function has a quadratic Taylor expansion near an optimum. 

$iii)$Hessian matrix of second derivatives does not need to be evaluated directly. Fast for large unconstrained optimization problems with a dense Hessian.

$iv)$Approximations may occasionally become very small, and the inverses of the approximations become very large, which could make the algorithm unstable. Storing the approximation in memory can be costly if n is large.


### Problem 7
write a function that computes the gradient $Df$ numerically using foreward differences.

In [5]:
import numpy as np

In [32]:
def grad(f, x_0):
    '''
        dim is the dimensionallity of the function.
    '''
    dim = len(x_0)
    E = np.identity(dim)
    h = 1e-6
    f0 = f(x_0)
    
    Df = np.zeros(dim)
    for i in range(dim):
        e = E[i]
        x_h = x_0 + h*e
        f_xh = f(x_h)
        D_i = (f_xh - f0) / h
        Df[i] = D_i
        
    return Df
    

In [33]:
Q = np.array([[1, 2], [3, 4]])
b = np.array([5, 6])
F = lambda x: 0.5 * x.T @ Q @ x - b.T @ x
x_0 = np.array([1, 1])


grad(F, x_0)

array([-1.4999995,  0.500002 ])

In [34]:
# analytical gradient
grad_anal = 0.5* (Q.T + Q)@ x_0 - b
grad_anal

array([-1.5,  0.5])

###  Problem 6
implementing the steepest decent method

In [41]:
import numpy as np

#Quadratic function which takes in matrix Q, vectors x, b, and c and returns the value of the function evaluated at x
def f(Q, x, b, c):
    return (1/2) * x.T @ Q @ x - b.T @ x + c

def grad(f, Q, x_0, b, c):
    dim = len(x_0)
    E = np.identity(dim)
    h = 1e-6
    f0 = f(Q, x_0, b, c)

    Df = np.zeros(dim)
    for i in range(dim):
        e = E[i]
        x_h = x_0 + h*e
        f_xh = f(Q, x_h, b, c)
        D_i = (f_xh - f0) / h
        Df[i] = D_i

    return Df

def graddesc(f, Q, x_init, b, c, eps = 1e-6):
    cur_x = np.copy(x_init)
    diff = 5
    while (diff > eps):
        prev_x = np.copy(cur_x)
        alpha = (grad(f, Q, prev_x, b, c) @ grad(f, Q, prev_x, b, c).T)/ ((grad(f, Q, prev_x, b, c)) @ Q @ grad(f, Q, prev_x, b, c).T)
        cur_x = cur_x - alpha * (grad(f, Q, prev_x, b, c).T)
        diff = np.linalg.norm(cur_x - prev_x)

    print("The function is minimized at ", cur_x)
    return

Q = np.array([[1,0], [0,1]])
b = np.array([0,0])
c = 0
x = np.array([2,5])

graddesc(f, Q, x, b, c, 1e-8)


The function is minimized at  [ -5.00000000e-07  -5.00000000e-07]


### Problem 10

Show that Newton's Method converges in one step for the function $f(x) = \frac{1}{2}x^TQx - b^Tx$ where Q is symmetric and psitive definite. We have the gradient of f $ Df(x) = \frac{1}{2}(Q^T + Q)x - b$ and the hessian of f $D^2f = \frac{1}{2}(Q^T + Q) $ so Newton's Method is given by :

$$ x_{k+1} = x_k - D^2f^{-1}Df(x_k) $$ 

$$ x_{k+1} = x_k -  \left[\frac{1}{2}(Q^T + Q) \right]^{-1} \left[\frac{1}{2}(Q^T + Q)x_k - b \right] $$ 

$$ x_{k+1} = x_k - 2(Q^T + Q)^{-1} \frac{1}{2}(Q^T + Q)x_k +2(Q^T + Q)^{-1} b  $$

$$ x_{k+1} = x_k - x_k +2(Q^T + Q)^{-1} b  $$  

so $x_k$ cancels out regradless of the number of iterations and we are leftt with 
$$x_{k+1} = +2(Q^T + Q)^{-1} b  $$ 